In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
housing = pd.read_csv('/kaggle/input/california-housing-prices/housing.csv')

In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
%matplotlib inline
housing.hist(bins = 50 , figsize = (20,15))
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train ,test = train_test_split(housing,test_size = 0.2,random_state = 42) 

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train['median_income'].hist()
plt.show()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
housing.plot(kind = 'scatter',x = 'longitude',y = 'latitude',color = 'k',alpha = 0.1)
plt.show()

In [ ]:
housing.plot(kind = 'scatter',x = 'longitude', y = 'latitude', alpha = 0.4,figsize = (10,10), s = housing['population']/100,label = 'population', c = 'median_house_value',cmap = plt.get_cmap('jet'),colorbar = True)
plt.legend()
plt.show()

In [ ]:
cor = housing.corr()

In [ ]:
cor['median_house_value']

In [ ]:
x = housing['median_income']
y = housing['median_house_value']

In [ ]:
x.size,y.size

In [ ]:
plt.scatter(x,y)
plt.show()

In [ ]:
housing['rooms_per_households'] = housing['total_rooms']/housing['households']
housing['population_per_households'] = housing['population']/housing['households']   
housing['bedrooms_per_room'] = housing['total_bedrooms']/housing['total_rooms'] 

In [ ]:
cor = housing.corr()

In [ ]:
cor['median_house_value']

In [ ]:
housing.head()

In [ ]:
housing = train.drop('median_house_value',axis = 1)
housing_labels = train['median_house_value'].copy()

In [ ]:
housing.info()

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
test_housing = test.drop('median_house_value',axis = 1)
test_labels = test['median_house_value'].copy() 

In [ ]:
type(test_housing),type(test_labels),type(housing)

In [ ]:
housing_labels.shape,test_labels.shape

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer = SimpleImputer(strategy = 'median')

In [ ]:
housing_num = housing.drop('ocean_proximity',axis = 1)
test_housing = test_housing.drop('ocean_proximity',axis = 1)

In [ ]:
imputer.fit(housing_num)
imputer.fit(test_housing)

In [ ]:
imputer.statistics_

In [ ]:
x = imputer.transform(test_housing)

In [ ]:
X = imputer.transform(housing_num)
test_housing = imputer.transform(test_housing)

In [ ]:
housing_transformed = pd.DataFrame(X,columns = housing_num.columns)
test_housing = pd.DataFrame(x,columns = housing_num.columns)

In [ ]:
housing_transformed.info()

In [ ]:
test_housing.info()

In [ ]:
housing_cat = housing["ocean_proximity"]
test_cat = test['ocean_proximity']

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
housing_catencoded = encoder.fit_transform(housing_cat)
housing_catencoded

In [ ]:
test_catencoded = encoder.fit_transform(test_cat)

In [ ]:
housing_catencoded.shape,test_catencoded.shape

In [ ]:
test_catencoded

In [ ]:
from sklearn.preprocessing import StandardScaler
stdscl = StandardScaler()
housing_prepared = stdscl.fit_transform(housing_transformed)
test_prepared = stdscl.fit_transform(test_housing)

In [ ]:
housing_prepared = pd.DataFrame(housing_prepared,columns = housing_transformed.columns)
test_prepared = pd.DataFrame(test_prepared,columns = housing_transformed.columns)

In [ ]:
housing_prepared['ocean_proximity'] = housing_catencoded

In [ ]:
housing_prepared.head()

In [ ]:
test_prepared['ocean_proximity'] = test_catencoded

In [ ]:
test_prepared.info()

In [ ]:
from sklearn.linear_model import LinearRegression
linr = LinearRegression()

In [ ]:
 linr.fit(housing_prepared,housing_labels)

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = linr.predict(housing_prepared)
err = mean_squared_error(housing_labels,housing_predictions)

In [ ]:
lin_rmse = np.sqrt(err)
lin_rmse

In [ ]:
from sklearn.metrics import mean_absolute_error
abserr = mean_absolute_error(housing_labels,housing_predictions)
abserr

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dcs = DecisionTreeRegressor()

In [ ]:
dcs.fit(housing_prepared,housing_labels)

In [ ]:
housing_predictions = dcs.predict(housing_prepared)

In [ ]:
from sklearn.metrics import mean_squared_error
dcs_reg = mean_squared_error(housing_labels,housing_predictions)

In [ ]:
dcs_reg = np.sqrt(dcs_reg)

In [ ]:
dcs_reg

In [ ]:
from sklearn.model_selection import cross_val_score
dcscore = cross_val_score(dcs,housing_prepared,housing_labels,cv =10,scoring = 'neg_mean_squared_error')

In [ ]:
rmse_Scores = np.sqrt(-dcscore)

In [ ]:
rmse_Scores

In [ ]:
print(rmse_Scores.mean())
print(rmse_Scores.std())

In [ ]:
lin_scores = cross_val_score(linr,housing_prepared,housing_labels,cv = 10, scoring = 'neg_mean_squared_error')

In [ ]:
lin_scores = np.sqrt(-lin_scores)

In [ ]:
print(lin_scores)
print(lin_scores.mean())
print(lin_scores.std())

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rndmfrst = RandomForestRegressor()

In [ ]:
rndmfrst.fit(housing_prepared,housing_labels)

In [ ]:
housing_predictions = rndmfrst.predict(housing_prepared)

In [ ]:
housing_predictions

In [ ]:
rndmerr = mean_squared_error(housing_labels,housing_predictions)

In [ ]:
rndmerr = np.sqrt(rndmerr)

In [ ]:
print(rndmerr)
print(rndmerr.mean())
print(rndmerr.std())

In [ ]:
rndmscores = cross_val_score(rndmfrst,housing_prepared,housing_labels,cv = 10,scoring = 'neg_mean_squared_error')

In [ ]:
rndmscores = np.sqrt(-rndmscores)
print(rndmscores.mean())
print(rndmscores)
print(rndmscores.std())

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},{'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}]
grid_search = GridSearchCV(rndmfrst,param_grid,cv = 5,scoring = 'neg_mean_squared_error')

In [ ]:
grid_search.fit(housing_prepared,housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
print(np.sqrt(-grid_search.best_score_))

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_

In [ ]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)


In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_

In [ ]:
feature_importances

In [ ]:
attribs = list(housing_prepared.columns)

In [ ]:
 sorted(zip(feature_importances, attribs), reverse=True)

In [ ]:
final_model = grid_search.best_estimator_

In [ ]:
final_predictions = final_model.predict(test_prepared) 

In [ ]:
final_mse = mean_squared_error(test_labels,final_predictions)

In [ ]:
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse